# A Large 3-SAT Grover search Example

In [1]:
EXPRESSION = """
((x2) or (x3) or (x4)) and
((not x1) or ( x2) or ( x3)) and
((not x1) or ( x2) or (not x3)) and
((not x1) or (not x2) or ( x3)) and
(( x1) or (not x2) or (not x3)) and
(( x1) or (not x2) or ( x3)) and
((not x1) or (not x2) or (not x4)) and
((not x1) or (not x2) or ( x4)) and
((not x2) or (not x3) or (not x4)) and
(( x2) or (not x3) or ( x4)) and
(( x1) or (not x3) or ( x4)) and
(( x1) or (not x2) or (not x4)) and
((not x1) or (not x2) or (not x3))
"""

In [2]:
try:
    import ttg

    print(
        ttg.Truths(
            ["x1", "x2", "x3", "x4"],
            [EXPRESSION[1:-1]],
        )
    )
except:
    print("Please import 'truth-table-generator' in order to print the truth table")

+------+------+------+------+------------------------------------------+
|  x1  |  x2  |  x3  |  x4  |        ((x2) or (x3) or (x4)) and        |
|      |      |      |      |     ((not x1) or ( x2) or ( x3)) and     |
|      |      |      |      |   ((not x1) or ( x2) or (not x3)) and    |
|      |      |      |      |   ((not x1) or (not x2) or ( x3)) and    |
|      |      |      |      |   (( x1) or (not x2) or (not x3)) and    |
|      |      |      |      |     (( x1) or (not x2) or ( x3)) and     |
|      |      |      |      |  ((not x1) or (not x2) or (not x4)) and  |
|      |      |      |      |   ((not x1) or (not x2) or ( x4)) and    |
|      |      |      |      |  ((not x2) or (not x3) or (not x4)) and  |
|      |      |      |      |     (( x2) or (not x3) or ( x4)) and     |
|      |      |      |      |     (( x1) or (not x3) or ( x4)) and     |
|      |      |      |      |   (( x1) or (not x2) or (not x4)) and    |
|      |      |      |      |    ((not x1) or (not 

## Loading the model

In [3]:
from classiq import RegisterUserInput, construct_grover_model

register_size = RegisterUserInput(size=1)

qmod = construct_grover_model(
    num_reps=1,
    expression="(" + EXPRESSION + ")",
    definitions=[
        ("x1", register_size),
        ("x2", register_size),
        ("x3", register_size),
        ("x4", register_size),
    ],
)

In [4]:
with open("3_sat_grover_large.qmod", "w") as f:
    f.write(qmod)

## Synthesizing the Circuit

We proceed by synthesizing the circuit using Classiq's synthesis engine. The synthesis should take approximately several seconds:

In [5]:
from classiq import GeneratedCircuit, synthesize

qprog = synthesize(qmod)

## Showing the Resulting Circuit

After Classiq's synthesis engine has finished the job, we can show the resulting circuit in the interactive GUI:

In [6]:
circuit = GeneratedCircuit.from_qprog(qprog)
circuit.show()

Opening: https://platform.classiq.io/circuit/8e0abc86-f473-4bf7-b651-e7cabdfcbcc6?version=0.34.0.dev0%2B124706fb0e


In [7]:
print(circuit.transpiled_circuit.depth)

1219


## Executing the circuit

Lastly, we can execute the resulting circuit with Classiq's execute interface, using the `execute` function. We select the number of iterations we wish to run (in this case - 1), and the execution backend (in this case - the IBM Aer simulator):

In [8]:
from classiq import execute, set_quantum_program_execution_preferences
from classiq.execution import (
    ClassiqBackendPreferences,
    ExecutionDetails,
    ExecutionPreferences,
)

backend_preferences = ExecutionPreferences(
    backend_preferences=ClassiqBackendPreferences(backend_name="aer_simulator")
)

qprog = set_quantum_program_execution_preferences(qprog, backend_preferences)
optimization_result = execute(qprog).result()

In [9]:
res = optimization_result[0].value

Printing out the result, we see that our execution of Grover's algorithm successfully found the satisfying assignments for the input formula with a high probability:

In [10]:
res.counts_of_multiple_outputs(("x1", "x2", "x3", "x4"))

{('1', '0', '1', '1'): 11,
 ('0', '0', '0', '0'): 14,
 ('1', '1', '1', '1'): 15,
 ('1', '0', '0', '0'): 17,
 ('0', '0', '1', '1'): 418,
 ('1', '0', '1', '0'): 13,
 ('1', '0', '0', '1'): 16,
 ('1', '1', '0', '1'): 17,
 ('1', '1', '1', '0'): 15,
 ('1', '1', '0', '0'): 18,
 ('0', '0', '0', '1'): 379,
 ('0', '1', '1', '0'): 13,
 ('0', '1', '1', '1'): 14,
 ('0', '0', '1', '0'): 16,
 ('0', '1', '0', '0'): 10,
 ('0', '1', '0', '1'): 14}